# 미니프로젝트: 단어 검색기

## 프로젝트 목표
- 함수를 활용한 실용적인 프로그램 개발
- 내장 함수와 람다 함수 적극 활용
- 텍스트 검색 및 분석 기능 구현
- 사용자 친화적인 인터페이스 제공

## 기능 요구사항
1. 단어 데이터베이스 관리
2. 다양한 검색 옵션 (정확 검색, 부분 검색, 패턴 검색)
3. 검색 결과 정렬 및 필터링
4. 단어 통계 분석
5. 북마크 기능

## 1. 기본 단어 검색기

In [ ]:
# 기본 단어 데이터베이스
word_database = [
    {'word': 'apple', 'meaning': '사과', 'category': 'fruit', 'difficulty': 1},
    {'word': 'banana', 'meaning': '바나나', 'category': 'fruit', 'difficulty': 1},
    {'word': 'computer', 'meaning': '컴퓨터', 'category': 'technology', 'difficulty': 2},
    {'word': 'programming', 'meaning': '프로그래밍', 'category': 'technology', 'difficulty': 3},
    {'word': 'beautiful', 'meaning': '아름다운', 'category': 'adjective', 'difficulty': 2},
    {'word': 'elephant', 'meaning': '코끼리', 'category': 'animal', 'difficulty': 2},
    {'word': 'python', 'meaning': '파이썬(뱀/언어)', 'category': 'technology', 'difficulty': 2},
    {'word': 'dictionary', 'meaning': '사전', 'category': 'object', 'difficulty': 3}
]

def search_exact(word, database=word_database):
    """정확한 단어 검색"""
    results = list(filter(lambda x: x['word'].lower() == word.lower(), database))
    return results

def search_partial(keyword, database=word_database):
    """부분 검색"""
    keyword = keyword.lower()
    results = list(filter(lambda x: keyword in x['word'].lower() or keyword in x['meaning'].lower(), database))
    return results

def search_by_category(category, database=word_database):
    """카테고리별 검색"""
    results = list(filter(lambda x: x['category'].lower() == category.lower(), database))
    return results

def display_results(results):
    """검색 결과 출력"""
    if not results:
        print("검색 결과가 없습니다.")
        return
    
    print(f"\n검색 결과 ({len(results)}개):")
    print("-" * 50)
    for word_info in results:
        print(f"단어: {word_info['word']}")
        print(f"뜻: {word_info['meaning']}")
        print(f"분류: {word_info['category']} (난이도: {word_info['difficulty']})")
        print("-" * 30)

# 기본 검색 테스트
print("=== 기본 단어 검색기 테스트 ===")
display_results(search_exact('apple'))
display_results(search_partial('com'))
display_results(search_by_category('fruit'))

## 2. 고급 검색 기능

In [ ]:
import re

class WordSearcher:
    def __init__(self, database=None):
        self.database = database or word_database
        self.bookmarks = []
        self.search_history = []
    
    def add_word(self, word, meaning, category, difficulty=1):
        """새 단어 추가"""
        new_word = {
            'word': word.lower(),
            'meaning': meaning,
            'category': category.lower(),
            'difficulty': difficulty
        }
        self.database.append(new_word)
        return f"'{word}' 단어가 추가되었습니다."
    
    def search_pattern(self, pattern):
        """정규표현식 패턴 검색"""
        try:
            regex = re.compile(pattern, re.IGNORECASE)
            results = list(filter(lambda x: regex.search(x['word']), self.database))
            self._add_to_history('pattern', pattern, len(results))
            return results
        except re.error:
            return []
    
    def search_advanced(self, **criteria):
        """고급 검색 (여러 조건 조합)"""
        results = self.database
        
        if 'word' in criteria:
            keyword = criteria['word'].lower()
            results = list(filter(lambda x: keyword in x['word'].lower(), results))
        
        if 'meaning' in criteria:
            keyword = criteria['meaning'].lower()
            results = list(filter(lambda x: keyword in x['meaning'].lower(), results))
        
        if 'category' in criteria:
            category = criteria['category'].lower()
            results = list(filter(lambda x: x['category'] == category, results))
        
        if 'min_difficulty' in criteria:
            min_diff = criteria['min_difficulty']
            results = list(filter(lambda x: x['difficulty'] >= min_diff, results))
        
        if 'max_difficulty' in criteria:
            max_diff = criteria['max_difficulty']
            results = list(filter(lambda x: x['difficulty'] <= max_diff, results))
        
        self._add_to_history('advanced', str(criteria), len(results))
        return results
    
    def sort_results(self, results, sort_by='word', reverse=False):
        """검색 결과 정렬"""
        sort_functions = {
            'word': lambda x: x['word'],
            'meaning': lambda x: x['meaning'],
            'category': lambda x: x['category'],
            'difficulty': lambda x: x['difficulty'],
            'length': lambda x: len(x['word'])
        }
        
        if sort_by in sort_functions:
            return sorted(results, key=sort_functions[sort_by], reverse=reverse)
        return results
    
    def add_bookmark(self, word_info):
        """북마크 추가"""
        if word_info not in self.bookmarks:
            self.bookmarks.append(word_info)
            return f"'{word_info['word']}' 북마크에 추가됨"
        return f"'{word_info['word']}' 이미 북마크에 있음"
    
    def get_bookmarks(self):
        """북마크 목록 반환"""
        return self.bookmarks
    
    def _add_to_history(self, search_type, query, result_count):
        """검색 기록 추가"""
        self.search_history.append({
            'type': search_type,
            'query': query,
            'results': result_count,
            'timestamp': f"검색_{len(self.search_history)+1}"
        })
    
    def get_statistics(self):
        """데이터베이스 통계"""
        total_words = len(self.database)
        categories = list(set(word['category'] for word in self.database))
        
        category_counts = {}
        difficulty_counts = {}
        
        for word in self.database:
            cat = word['category']
            diff = word['difficulty']
            category_counts[cat] = category_counts.get(cat, 0) + 1
            difficulty_counts[diff] = difficulty_counts.get(diff, 0) + 1
        
        avg_length = sum(len(word['word']) for word in self.database) / total_words
        
        return {
            'total_words': total_words,
            'categories': len(categories),
            'category_distribution': category_counts,
            'difficulty_distribution': difficulty_counts,
            'average_word_length': round(avg_length, 2),
            'search_count': len(self.search_history)
        }

# 고급 검색기 테스트
searcher = WordSearcher()

print("=== 고급 검색 기능 테스트 ===")

# 패턴 검색
pattern_results = searcher.search_pattern(r'^a')
print(f"'a'로 시작하는 단어: {[w['word'] for w in pattern_results]}")

# 고급 검색
advanced_results = searcher.search_advanced(category='technology', min_difficulty=2)
print(f"기술 분야 난이도 2 이상: {[w['word'] for w in advanced_results]}")

# 결과 정렬
sorted_by_length = searcher.sort_results(searcher.database, 'length', reverse=True)
print(f"길이순 정렬: {[w['word'] for w in sorted_by_length[:3]]}")

# 통계
stats = searcher.get_statistics()
print(f"\n데이터베이스 통계: {stats}")

## 3. 완전한 단어 검색 시스템

In [ ]:
def create_word_search_system():
    """완전한 단어 검색 시스템"""
    
    # 확장된 단어 데이터베이스
    extended_database = word_database + [
        {'word': 'function', 'meaning': '함수', 'category': 'programming', 'difficulty': 2},
        {'word': 'lambda', 'meaning': '람다', 'category': 'programming', 'difficulty': 3},
        {'word': 'algorithm', 'meaning': '알고리즘', 'category': 'programming', 'difficulty': 3},
        {'word': 'variable', 'meaning': '변수', 'category': 'programming', 'difficulty': 2},
        {'word': 'database', 'meaning': '데이터베이스', 'category': 'technology', 'difficulty': 3}
    ]
    
    searcher = WordSearcher(extended_database)
    
    # 시뮬레이션된 사용자 인터랙션
    simulation_commands = [
        ('search', 'python'),
        ('pattern', r'.*ing$'),
        ('category', 'programming'),
        ('advanced', {'category': 'technology', 'max_difficulty': 2}),
        ('stats', None),
        ('bookmark', 'python'),
        ('quit', None)
    ]
    
    print("🔍 단어 검색 시스템 시작")
    print("=" * 50)
    
    for command, param in simulation_commands:
        print(f"\n명령: {command} {param if param else ''}")
        
        if command == 'search':
            results = list(filter(lambda x: param.lower() in x['word'].lower(), searcher.database))
            print(f"'{param}' 검색 결과: {len(results)}개")
            for result in results:
                print(f"  - {result['word']}: {result['meaning']}")
        
        elif command == 'pattern':
            results = searcher.search_pattern(param)
            print(f"패턴 '{param}' 검색 결과: {len(results)}개")
            for result in results:
                print(f"  - {result['word']}: {result['meaning']}")
        
        elif command == 'category':
            results = searcher.search_advanced(category=param)
            print(f"카테고리 '{param}' 검색 결과: {len(results)}개")
            for result in results:
                print(f"  - {result['word']}: {result['meaning']}")
        
        elif command == 'advanced':
            results = searcher.search_advanced(**param)
            print(f"고급 검색 결과: {len(results)}개")
            for result in results:
                print(f"  - {result['word']}: {result['meaning']} (난이도: {result['difficulty']})")
        
        elif command == 'stats':
            stats = searcher.get_statistics()
            print("📊 데이터베이스 통계:")
            print(f"  전체 단어 수: {stats['total_words']}")
            print(f"  카테고리 수: {stats['categories']}")
            print(f"  평균 단어 길이: {stats['average_word_length']}")
            print(f"  카테고리별 분포: {stats['category_distribution']}")
        
        elif command == 'bookmark':
            word_info = next((w for w in searcher.database if w['word'] == param), None)
            if word_info:
                message = searcher.add_bookmark(word_info)
                print(f"🔖 {message}")
            else:
                print(f"'{param}' 단어를 찾을 수 없습니다.")
        
        elif command == 'quit':
            print("\n시스템을 종료합니다.")
            print(f"검색 기록: {len(searcher.search_history)}개")
            print(f"북마크: {len(searcher.bookmarks)}개")
            break
    
    return searcher

# 시스템 실행
final_searcher = create_word_search_system()

## 4. 분석 및 추천 기능

In [ ]:
def analyze_learning_progress(searcher):
    """학습 진도 분석"""
    
    # 검색 기록 분석
    search_patterns = {}
    for record in searcher.search_history:
        search_type = record['type']
        search_patterns[search_type] = search_patterns.get(search_type, 0) + 1
    
    # 북마크된 단어의 난이도 분석
    bookmark_difficulties = [word['difficulty'] for word in searcher.bookmarks]
    avg_bookmark_difficulty = sum(bookmark_difficulties) / len(bookmark_difficulties) if bookmark_difficulties else 0
    
    # 추천 난이도 계산
    if avg_bookmark_difficulty < 1.5:
        recommended_difficulty = 2
        level = "초급"
    elif avg_bookmark_difficulty < 2.5:
        recommended_difficulty = 3
        level = "중급"
    else:
        recommended_difficulty = 3
        level = "고급"
    
    return {
        'search_patterns': search_patterns,
        'avg_bookmark_difficulty': round(avg_bookmark_difficulty, 2),
        'recommended_difficulty': recommended_difficulty,
        'level': level
    }

def recommend_words(searcher, count=5):
    """단어 추천 시스템"""
    
    analysis = analyze_learning_progress(searcher)
    target_difficulty = analysis['recommended_difficulty']
    
    # 북마크되지 않은 단어 중에서 추천
    bookmarked_words = {word['word'] for word in searcher.bookmarks}
    
    candidates = list(filter(
        lambda w: w['word'] not in bookmarked_words and w['difficulty'] == target_difficulty,
        searcher.database
    ))
    
    # 단어 길이와 카테고리 다양성을 고려하여 정렬
    scored_candidates = []
    for word in candidates:
        # 점수 계산: 적당한 길이(5-8자)에 높은 점수
        length_score = 10 - abs(len(word['word']) - 6.5)
        scored_candidates.append((word, length_score))
    
    # 점수순으로 정렬
    scored_candidates.sort(key=lambda x: x[1], reverse=True)
    
    return [word for word, score in scored_candidates[:count]]

def generate_quiz(searcher, count=3):
    """퀴즈 생성"""
    import random
    
    # 북마크된 단어에서 퀴즈 생성
    if len(searcher.bookmarks) < count:
        quiz_words = searcher.bookmarks + random.sample(
            [w for w in searcher.database if w not in searcher.bookmarks],
            count - len(searcher.bookmarks)
        )
    else:
        quiz_words = random.sample(searcher.bookmarks, count)
    
    quiz_questions = []
    for word_info in quiz_words:
        # 객관식 선택지 생성
        correct_answer = word_info['meaning']
        wrong_answers = [w['meaning'] for w in searcher.database 
                        if w['meaning'] != correct_answer and w['category'] == word_info['category']]
        
        if len(wrong_answers) >= 3:
            choices = [correct_answer] + random.sample(wrong_answers, 3)
        else:
            choices = [correct_answer] + wrong_answers
            while len(choices) < 4:
                filler = random.choice([w['meaning'] for w in searcher.database])['meaning']
                if filler not in choices:
                    choices.append(filler)
        
        random.shuffle(choices)
        correct_index = choices.index(correct_answer)
        
        quiz_questions.append({
            'word': word_info['word'],
            'choices': choices,
            'correct_answer': correct_index,
            'difficulty': word_info['difficulty']
        })
    
    return quiz_questions

# 분석 및 추천 테스트
print("=== 학습 분석 및 추천 시스템 ===")

# 학습 진도 분석
progress = analyze_learning_progress(final_searcher)
print(f"학습 분석 결과:")
print(f"  검색 패턴: {progress['search_patterns']}")
print(f"  평균 북마크 난이도: {progress['avg_bookmark_difficulty']}")
print(f"  추천 레벨: {progress['level']}")

# 단어 추천
recommendations = recommend_words(final_searcher)
print(f"\n📚 추천 단어 ({len(recommendations)}개):")
for word in recommendations:
    print(f"  - {word['word']}: {word['meaning']} (난이도: {word['difficulty']})")

# 퀴즈 생성
quiz = generate_quiz(final_searcher)
print(f"\n❓ 생성된 퀴즈 ({len(quiz)}문):")
for i, question in enumerate(quiz, 1):
    print(f"  {i}. '{question['word']}'의 뜻은?")
    for j, choice in enumerate(question['choices']):
        marker = "★" if j == question['correct_answer'] else " "
        print(f"     {j+1}{marker} {choice}")

## 정리

이번 미니프로젝트에서 구현한 기능들:

### 🔍 핵심 기능
1. **기본 검색**: 정확 검색, 부분 검색, 카테고리 검색
2. **고급 검색**: 패턴 검색, 다중 조건 검색
3. **결과 처리**: 정렬, 필터링, 페이징
4. **사용자 기능**: 북마크, 검색 기록

### 📊 분석 기능
1. **통계 분석**: 데이터베이스 현황, 사용 패턴
2. **학습 분석**: 진도 추적, 레벨 판정
3. **추천 시스템**: 맞춤형 단어 추천
4. **퀴즈 생성**: 자동 문제 출제

### 🛠️ 사용된 함수 기법
1. **람다 함수**: 필터링, 정렬, 변환
2. **내장 함수**: `filter()`, `map()`, `sorted()`
3. **고차 함수**: 함수를 인수로 받는 함수
4. **클로저**: 상태를 유지하는 함수

### 💡 학습 포인트
- 함수형 프로그래밍 패러다임 활용
- 데이터 처리 파이프라인 구성
- 객체지향과 함수형의 조화
- 실제 서비스 수준의 기능 구현

이제 Chapter06 함수편이 완료되었습니다! 🎉
다음 Chapter07에서는 객체지향 프로그래밍을 시작하겠습니다.